Получаем данные.

In [1]:
import random

objectList=[]
dataFile=open('16.txt', 'r')
for line in dataFile:
    objectList.append(line.split())
dataFile.close()
maxWeight=float(objectList[0][0])
maxVolume=float(objectList[0][1])
objectList.pop(0)
numberOfObjects=len(objectList)
parentsIndexes=[]
solutions=[]
selected=[]
parents=[]
populationSize=200
best=0
bestSolution=[]


Получаем начальную популяцию - 200 особей, каждая особь - битовая последовательность длиной 30 (количество предметов в списке), особи генерируются случайно.

In [2]:
import random
def solutionsGenerator(length, finish):
    start=0
    while start<finish:
        res=[]
        for i in range(0, length):
            res.append(random.randrange(2))
        start+=1
        yield res

        
for i in solutionsGenerator(numberOfObjects, populationSize):
    solutions.append(i)

Выбираем особи для скрещивания. Выбор производится методом рулетки. Вычислим функцию приспособленности для каждой особи, в зависимости от значения функции приспособленности вычислим вероятность выбора данной особи, и на основе вычисленных вероятностей выберем 20% особей.    
Функция приспособленности - суммарная ценность предметов при условии, что суммарные вес и объем предметов не превышают заданные грузоподъемность и вместительность.

In [3]:
#ищем значения функции приспособленности
def fitness(k):
    global solutions
    global best
    global bestSolution
    
    summValue=0
    summVolume=0
    summWeight=0
    for i in range (0, len(solutions[k])):
        summWeight+=solutions[k][i]*float(objectList[i][0])
        summVolume+=solutions[k][i]*float(objectList[i][1])
        summValue+=solutions[k][i]*float(objectList[i][2])
    if summVolume<=maxVolume and summWeight<=maxWeight:
        res=summValue
    else:
        res=0
    if res>best:
        best=res
        bestSolution=solutions[k]
    return res

    
#ищем вероятности    
def prob():
    global solutions
    global best
    best=0
    summFitness=0
    for i in range(0, len(solutions)):
        fit=fitness(i)
        summFitness+=fit
        solutions[i].append(fit)
    for i in range(0, len(solutions)):
        solutions[i].append(solutions[i][-1]/summFitness)
        
#ищем концы отрезков на "рулетке"        
def sections():
    global solutions
    current=0
    for solution in solutions:
        if solution[-1]==0:
            solution.append(0)
            solution.append(0)
        else:
            solution.append(current)
            solution.append(current+solution[-2])
            current+=solution[-3]        

In [4]:
def selection():
    global solutions
    global selected
    global parentsIndexes
    parentsIndexes=[]
    selected=[]
  
    counter=0
    while counter<int(populationSize*0.2):
        rand=random.random()
        for j in range(0, len(solutions)):
            if solutions[j][-2]<=rand and solutions[j][-1]>=rand:
                if j not in parentsIndexes:
                    counter+=1
                    sol=[]
                    for k in range(0, numberOfObjects):
                        sol.append(solutions[j][k])
                    selected.append(sol)
                    parentsIndexes.append(j)
                    
    for i in range (0, len(solutions)):
        sol=solutions[i]
        solutions[i]=[]
        for j in range(0, numberOfObjects):
            solutions[i].append(sol[j])

Скрещиваем особей. Каждая особь скрещивается один раз, каждая пара дает два потомка. Потомок получает каждый бит случайным образом от одного из родителей.

In [5]:
def crossOver():
    global selected
    global parents
    parents=[]
    for i in selected:
        i.append(0)
    for i in range(0, len(selected)):
        if selected[i][-1]==0:
            crossedOver=0
            while crossedOver==0:
                r=random.randrange(int(populationSize*0.2))
                if selected[r][-1]==0:
                    crossedOver=1
            firstChild=[]
            secondChild=[]
            for j in range(0, numberOfObjects):
                rand=random.randrange(2)
                if(rand==1):
                    firstChild.append(selected[i][j])
                else:
                    firstChild.append(selected[r][j])
                rand=random.randrange(2)
                if(rand==1):
                    secondChild.append(selected[i][j])
                else:
                    secondChild.append(selected[r][j])
            parents.append((selected[i], firstChild))
            parents.append((selected[r], secondChild))
            firstChild.append(1)
            secondChild.append(1)
            selected[i]=list(firstChild)
            selected[r]=list(secondChild)
    for i in selected:
        i.pop()

Мутация: инвертируем все биты у одной особи.

In [6]:
def mutate():
    global solutions
    rand=random.randrange(len(solutions))
    for i in range(0, numberOfObjects):
        if solutions[rand][i]==1:
            solutions[rand][i]=0
        else:
            solutions[rand][i]=1

Формирование нового поколения: меняем родителей на потомков.

In [7]:
def newPop():
    global solutions
    global selected
    k=0

    for j in range(0, len(solutions)):
        s=list(solutions[j])
        solutions[j]=[]
        for i in range(0, numberOfObjects):
            solutions[j].append(s[i])
    for j in range(0, len(selected)):
        s=list(selected[j])
        selected[j]=[]
        for i in range(0, numberOfObjects):
            selected[j].append(s[i])
    for i in parentsIndexes:
        if fit(solutions[i])<fit(selected[k]):
            solutions[i]=selected[k]
        k+=1           
        
            
def fit(sol):
    w=0
    vol=0
    val=0
    for i in range(0, numberOfObjects):
        w+=float(objectList[i][0])*sol[i]
        vol+=float(objectList[i][1])*sol[i]
        val+=float(objectList[i][2])*sol[i]
    if vol<=maxVolume and w<=maxWeight:
        return val
    else:
        return 0
        
#newPop()

In [8]:
counter=0
while counter<=500:    
    newsel=list(solutions)
    prob()
    sections()
    selection() 
    crossOver()  
    mutate()
    newPop()
    counter+=1
s=list(bestSolution)
bestSolution=[]
for i in range(0, numberOfObjects):
    bestSolution.append(s[i])
print(best)
print(bestSolution)
w=0
v=0
objects=''
for i in range(0, numberOfObjects):
    if bestSolution[i]!=0:
        objects+=objectList[i][0]+' '+objectList[i][1]+' '+objectList[i][2]+'\n'
    w+=round(bestSolution[i]*float(objectList[i][0]))
    v+=bestSolution[i]*float(objectList[i][1])
v=round(v, 1)
print(w)
print(v)



4452.0
[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
12933
11.9


In [9]:
f=open('res.txt', 'w')
res='Оптимальная цена: '+str(best)+'\n'+'Оптимальный вес: '+str(w)+'\n'+'Оптимальный объем: '+str(v)+'\n'+'Список объектов:\n'+objects
f.write(res)
f.close()